# Optimization of Material Flows in Matrix Production using QAOA

In this notebook, a matrix production material flow including 6 unique producing units/machines and three product types/material flows is simulated. For that purpose, an existing code construct by Madadi (2021) simulating a collision-free Vehicle Routing Problem (VRP) is partially transferred, amended and optimized to fulfill its purpose. 

The matrix-structured manufacturing system (MMS) simulated in this notebook is depicted below. As stated above, it consists of 6 machines that are feasible to excute three operations respectively. 

<img src="mms_layout_example.jpg" width="400" style="float:left"/>

The material flow directions and paths are not just chosen arbitrarily. The following image shows three different material flow schemes for three different product types. All products need exactly three manufacturing operations to be finished. Each operation can be executed by two machines. For example, the first operation of material flow 1 can be executed by machines *M1* and *M2*. These are combined in the previous image. The objective of this simple example is to find the optimum path through the MMS for each material flow without colliding at one machine. 

**Assumptions**: 
- The duration of moving a material from one node to another is equal for each distribution task.
- One time tick represents the material flow from one machine to another. 
- Therefore, the total time horizon is considering 5 time ticks (see further below).
- Processing times of machines are neglectable.
- At the starting time all machines are unoccupied and available.
- Buffers at machines are not existing. So, at most one material can be at one node at a time.

<img src="examplary_material_flows.jpg" width="800" style="float:left"/>

For the above material flows, a sliced graph can be constructed that shows the possible movement of material over each time tick t. The graph is depicted in the image below. The start-end relations are transferred into arrays in which one sub-array represents the feasible machines of one product for one respective time step. For example, the first operation of material flow 2 (colored purple) needs to be performed by machine *M2* or *M3*, the second operation by *M1* or *M6*, and the last by *M4* or *M5*. For each scenario, so for each unit in the array, one qubit is responsible. As you can see, there are a total of 24 qubits needed to simulate all three material flows. Qubits 1-8 represent material flow 1 (*MF1*), qubits 9-16 represent material flow 2 (*MF2*), and qubits 17-24 represent material flow 3 (*MF3*).

<img src="sliced_graph.jpg" width="800" style="float:left"/>

Each edge that connects two nodes/machines is assinged to a specific cost. A cost matrix is generated that includes all the costs of choosing a path between two nodes. The costs are chosen as follows: 
- a cost of 1 unit is assigned if the subsequent node is located in front or horizontal to the current node (e.g. S-M2, M1-M3, M5-M6); 
- a cost of 2 units is assigned if the subsequent node is located diagonal to the current node (e.g. M1-M4, M3-M6); 
- a cost of 3 units is assigned if the subsequent node is located behind the node in front of the current node (e.g. M2-M6, M3-D);
- a cost of P (penalty weight) is assigned if the the path is not present in the given example.


The underlying **QUBO** $H = H_c + H_1 + H_2$for the MMS Problem consists of one main cost function $H_c$ that is subject to two constraints $H_1$ and $H_2$.

$H_c$ iterates through every material flow at every time tick at every vertex and calculates the cost sum of all active vertices. One cost is accumulated by two active subsequent vertices v and v'. 

\begin{equation}
H_c = \sum_{m=0}^{M-1}\sum_{t=0}^{T-2}\sum_{v\in{V_t}, v'\in{V_{t+1}}}c_{v,v'}X_{m,t,v}X_{m,t+1,v'}
\end{equation}

$H_1$ iterates through every material flow at every time tick to ensure that only one vertex is active at one respective time tick. If this is not the case, a penalty weight P is accumulated to the total cost.

\begin{equation}
H_1 = P\sum_{m=0}^{M-1}\sum_{t=0}^{T-1}(\sum_{v\in{V_t}}X_{m,t,v}-1)^2
\end{equation} 

$H_2$ iterates through every time tick and every vertex to control if only one material flow is active on one vertex at a time. Function D's output is the number of material flows on one specific vertex. If a vertex includes more than 1 material flow at a time, a penalty weight P is accumulated as well. 

\begin{equation}
H_2 = P\sum_{t=1}^{T-2}\sum_{v\in{V}}(D(t,v)-1)D(t,v)
\end{equation} 

**Parameters**:
- $m$:   material flow
- $M$:   total number of material flows
- $t$:   time tick / time slice
- $T$:   total number of time ticks / time slices
- $v$:   vertex / node
- $v'$:  subsequent vertex / node of v
- $V_c$: set of all vertices / nodes
- $P$:   penalty value
- $X_{m,t,v}$: binary variable that is set to one if one specific node is active, otherwise it's value is 0

In the section "Preprocessing", three choices can be made. For testing reasons, two material flows (MF1 & MF2) have been simulated first. There are two scenarios you can choose from: 
- First Example - Simulation of 2 Material Flows with Multiple Optimal Solutions (see chapter 4.3.1)
- Second Example - Simulation of 2 Material Flows with 1 Optimal Solutions (see chapter 4.3.2)

For the simulation of three material flows, the third scenario has to be chosen:
- Third Example - Simulation of 3 Material Flows with Multiple Optimal Solutions (see chapter 4.3.3)

## Definition of Classes

In [1]:
def get_unique_nodes(edge_list):
    """
    Creates a list of all unique nodes appearing in a given MMS.
    The start node is added by default; every other node is only included if it can be reached from another node. 
    Isolated nodes will not be added to the list, unless they happen to be the start node.
    For the given example of an MMS with six workstations, this will always end up in the list [0,1,2,3,4,5,6,7] but
    for other examplary use cases (e.g. VRP) this might be important to consider.
    """
    
    nr_nodes = 0
    unique_nodes = []
    unique_nodes.append(start_node)

    for edge in edge_list:
        if edge[1] not in unique_nodes:
            unique_nodes.append(edge[1])

    return unique_nodes

In [2]:
def construct_numbered_sliced_MF(sliced_mf, offset):
    """
    Based on a two-dimensional array that contains the time ticks of an MMS, this function renumbers the contents by 
    giving each element its index in the corresponding flattened array. Since the numbered sliced MF is used for 
    getting the qubit indices of the nodes, and since there are several sliced material flows, an offset is necessary.
    Example: With input ([[0], [1, 3], [2, 3]], 0), output would be [[0], [1, 2], [3, 4]].
    With input ([[0], [1, 3], [2, 3], [4, 3]], 5), output would be [[5], [6, 7], [8, 9], [10, 11]].
    """
    
    ns = [] # numbered sliced material flow
    node_counter = 0

    for m in range(len(sliced_mf)):
        ns.append([])
        for n in range(len(sliced_mf[m])):
            ns[m].append(node_counter + offset)
            node_counter+=1
    
    return ns, node_counter

In [3]:
def construct_MF_matrix(edge_list):
    """
    Constructs a dataframe that contains all material flow edges present in the MMS. 
    The returned dataframe can contain several edges per two nodes. The entry at matrix[0][1] contains an array 
    of all edges that go from node 0 to node 1.
    """

    node_labels = get_unique_nodes(edge_list)
    MF_matrix = pd.DataFrame([[[] for _ in range(len(node_labels))] for _ in range(len(node_labels))], node_labels, node_labels)
    
    for edge in edge_list:
        start = edge[0]
        end = edge[1]
        MF_matrix[start][end].append(edge)
    
    return MF_matrix

In [4]:
def construct_MF_cost_matrix(flow_matrix):
    """
    Constructs a dataframe that contains the cost for the edge between any given pair of nodes a and b.
    Therefore, it takes the output of function construct_MF_matrix and assigns each edge a specific cost.
    If there is no outgoing edge from a to b, the cost will be equal to the fixed penalty value. 
    If there is exactly one edge, its cost will be used. If there are several, the smallest cost is used.
    """

    node_labels = flow_matrix.columns
    mf_cost_matrix = pd.DataFrame([[[] for _ in range(len(node_labels))] for _ in range(len(node_labels))], node_labels, node_labels)

    for i in node_labels:
        for j in node_labels:
            entries = flow_matrix[i][j]
            if len(entries) == 0:
                mf_cost_matrix[i][j] = (penalty_value)
            elif len(entries) == 1:
                mf_cost_matrix[i][j] = (entries[0][2])
            else:
                best_cost = penalty_value # penalty by default bigger than the biggest edge cost
                for entry in entries:
                    if entry[2] < best_cost:
                        best_cost = entry[2]
                mf_cost_matrix[i][j] = best_cost
    return mf_cost_matrix

In [5]:
def D_function(node, time_slice, nr_flows):
    """
    This function computes the sum over all binary variables associated to a given machine/node 
    which appear in the the same time tick of multiple material flows.
    """

    flows_at_position = 0
    for i in range(nr_flows):
        if len(mf_s[i]) > time_slice and node in mf_s[i][time_slice]:
            index = find_index(node, time_slice, mf_s[i], mf_ns[i])
            flows_at_position += X[index]

    return flows_at_position

In [6]:
def find_index(node, time_slice, sliced_mf, numbered_sliced_mf):
    """
    Determination of the qubit index for a given node. 
    If the node is not present in the given slice in the MMS, returns -1.
    """
    
    for i in range(len(sliced_mf[time_slice])):
        if sliced_mf[time_slice][i] == node:
            return numbered_sliced_mf[time_slice][i]
    else:
        return -1


In [7]:
def fix_nodes(start, destination, flow_index, sliced_mf, numbered_sliced_mf, cost_func):
    """
    This function fixes specific nodes in the cost function according to already known values for the qubits. 
    Here, the qubits representing the start and destination nodes are fixed at 1.
    """

    # set start node to 1
    start_node_index = 'X_' + str(find_index(start, 0, sliced_mf, numbered_sliced_mf))
    cost_func.linear_constraint(linear={start_node_index: 1}, sense='==', rhs=1, name=('start_node' + '_veh_' + str(flow_index)))

    # set destination node to 1
    dest_node_index = 'X_' + str(find_index(destination, -1, sliced_mf, numbered_sliced_mf))
    cost_func.linear_constraint(linear={dest_node_index: 1}, sense='==', rhs=1, name='destination_node' + '_veh_' + str(flow_index))

    return

In [8]:
def get_node_info_from_index(node_index, sliced_mf, numbered_sliced_mf):
    """ 
    For a given qubit index, returns the corresponding time tick and the node label. 
    """

    for time_slice in range(len(numbered_sliced_mf)):
        if node_index in numbered_sliced_MF[time_slice]:

            pos = numbered_sliced_mf[time_slice].index(node_index)
            node_label = sliced_mf[time_slice][pos]

            return time_slice, node_label

## Import of Libraries and Tools

In [9]:
# Import of all required libraries and tools to simulate the given problem.

# General Python libraries and tools
import sympy as sym
import pandas as pd
import time
from tqdm import tqdm

# Qiskit libraries and tools
import qiskit
from qiskit.algorithms import QAOA
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.utils import QuantumInstance
from qiskit_optimization.problems import QuadraticProgram
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, POWELL, SLSQP, SPSA

## Preprocessing

In [10]:
# There are several examples defined in the following. Each represents a given problem MMS as a collection of edges 
# with the format (start, end, edge weight) as well as defined start and destination nodes and an array of set manufacturing
# sequences for a number of products mf_s [[[start], [(operation 1) machine x, machine y], ..., [destination]], ...].
# Here, machine x and machine y correspond to 2 machines that have the skill to execute the needed operation.

# In general, the parameters i, j and k in mf_s[i][j][k] represent the material flow, product type or job number (i), 
# the operation number of the respective job (j) and the possible machine choice (k). The latter is processed in the qubits.

# Fixed variable assignments can be taken for the start and destination nodes, which are representing node 0 (start) and 
# node 7 (end) for each material flow defined in the material flow array mf_s further below.

start_node = 0
dest_node = 7

# Depending on which example you want to simulate, choose one of the following three sections and comment the other out. 
# If you all three uncommented, the last one is executed automatically.

# --- First Example - Simulation of 2 Material FLows with Multiple Optimal Solutions (see chapter 4.3.1) ---

MMS = [(0,1,1), (0,2,1), (0,3,3),
       (1,3,1), (1,4,2), (1,5,3),
       (2,1,1), (2,3,2), (2,4,1), (2,6,3),
       (3,1,1), (3,5,1), (3,6,2),
       (4,5,2), (4,6,1), (4,7,3),
       (5,7,1),
       (6,4,1), (6,5,1), (6,7,1)]

mf_s = [[[0],[1,2],[3,4],[5,6],[7]],
       [[0],[2,3],[1,6],[4,5],[7]]]

# --- Second Example - Simulation of 2 Material FLows with 1 Optimal Solution (see chapter 4.3.2) ---

MMS = [(0,1,1), (0,2,1), (0,3,7),
       (1,3,1), (1,4,2), (1,5,7),
       (2,1,1), (2,3,7), (2,4,7), (2,6,7),
       (3,1,7), (3,5,1), (3,6,7),
       (4,5,7), (4,6,7), (4,7,3),
       (5,7,1),
       (6,4,7), (6,5,7), (6,7,7)]

mf_s = [[[0],[1,2],[3,4],[5,6],[7]],
       [[0],[2,3],[1,6],[4,5],[7]]]

# --- Third Example - Simulation of 3 Material FLows with Multiple Optimal Solutions (see chapter 4.3.3) --- 

MMS = [(0,1,1), (0,2,1), (0,3,3), (0,4,3),
       (1,2,1), (1,3,1), (1,4,2), (1,5,3),
       (2,1,1), (2,3,2), (2,4,1), (2,6,3),
       (3,1,1), (3,5,1), (3,6,2), (3,7,3),
       (4,2,1), (4,5,2), (4,6,1), (4,7,3),
       (5,3,1), (5,6,1), (5,7,1),
       (6,4,1), (6,5,1), (6,7,1)]

mf_s = [[[0],[1,2],[3,4],[5,6],[7]],
       [[0],[2,3],[1,6],[4,5],[7]],
       [[0],[1,4],[2,5],[3,6],[7]]]


In [11]:
# Other variable definitions required for further steps

mf_s_list = []             # Format material flow array into list to allow later result formatting               
for sublist in mf_s:
    for sub_sublist in sublist:
        mf_s_list.extend(sub_sublist)
        
penalty_value = 200        # The penalty value that is impinged on a false/infeasible follow-up machine choosen

nr_flows = len(mf_s)       # Total number of material flows

unique_nodes = get_unique_nodes(MMS)    # List of all unique nodes in the MMS (start, M1, ..., M6, dest)

In [12]:
# Numbered and sliced material flows
mf_ns = []

nodes_total = 0
for i in range(nr_flows):
#   mf_s.append(construct_sliced_MF(mf_s_edges[i], start_node, dest_node))

    numbered_MMS, nr_nodes_MMS = construct_numbered_sliced_MF(mf_s[i], offset=nodes_total)
    mf_ns.append(numbered_MMS)

    nodes_total += nr_nodes_MMS

nr_qubits = nodes_total

print('Sliced MMS: \t\t' + str(mf_s))
print('Numbered Sliced MMS: \t' + str(mf_ns))
print('Qubits needed: \t\t' + str(nr_qubits))

Sliced MMS: 		[[[0], [1, 2], [3, 4], [5, 6], [7]], [[0], [2, 3], [1, 6], [4, 5], [7]]]
Numbered Sliced MMS: 	[[[0], [1, 2], [3, 4], [5, 6], [7]], [[8], [9, 10], [11, 12], [13, 14], [15]]]
Qubits needed: 		16


In [13]:
# Matrix that includes all edge costs for every feasible material flow route in the MMS.
# An edge cost of 0 is added for the destination in case one product type has fewer operations than others.

MMS_matrix = construct_MF_matrix(MMS)
MMS_matrix[7][7].append((7, 7, 0))
MMS_matrix

,0,1,2,3,4,5,6,7
0,[],[],[],[],[],[],[],[]
1,"[(0, 1, 1)]",[],"[(2, 1, 1)]","[(3, 1, 1)]",[],[],[],[]
2,"[(0, 2, 1)]",[],[],[],[],[],[],[]
3,"[(0, 3, 3)]","[(1, 3, 1)]","[(2, 3, 2)]",[],[],[],[],[]
4,[],"[(1, 4, 2)]","[(2, 4, 1)]",[],[],[],"[(6, 4, 1)]",[]
5,[],"[(1, 5, 3)]",[],"[(3, 5, 1)]","[(4, 5, 2)]",[],"[(6, 5, 1)]",[]
6,[],[],"[(2, 6, 3)]","[(3, 6, 2)]","[(4, 6, 1)]",[],[],[]
7,[],[],[],[],"[(4, 7, 3)]","[(5, 7, 1)]","[(6, 7, 1)]","[(7, 7, 0)]"


In [14]:
# Cost matrix including all edge costs for every (in)feasible material flow route in the MMS.
# The previously defined penalty value is resembling all routes that cannot be realized in the defined MMS problem. 

edge_cost_matrix = construct_MF_cost_matrix(MMS_matrix)
edge_cost_matrix

,0,1,2,3,4,5,6,7
0,200,200,200,200,200,200,200,200
1,1,200,1,1,200,200,200,200
2,1,200,200,200,200,200,200,200
3,3,1,2,200,200,200,200,200
4,200,2,1,200,200,200,1,200
5,200,3,200,1,2,200,1,200
6,200,200,3,2,1,200,200,200
7,200,200,200,200,3,1,1,0


In [15]:
# Definition of a variable that is assigned to the value of total time ticks one material flow requires to pass through 
# the MMS. The variable is later required as input for the D_function that iterates through each time tick and node to 
# control for collisions. In this project's problem definition, the time ticks needed are equal for every material flow. 
# But it might be changed if products are added, which need less or more than three operations to be manufactured. 

highest_slice_nr = 0
for i in range(nr_flows):
    if len(mf_s[i]) > highest_slice_nr:
        highest_slice_nr = len(mf_s[i])


## Definition of Variables

In [16]:
# Definition of all necessary sympy variables to be included in the upcoming cost function.

X = sym.IndexedBase('X') # counter variable for qubits
v = sym.symbols('v') # counter variable for vertices (nodes)
w = sym.symbols('w') # counter variable for subsequent vertices at t+1
k = sym.symbols('k') # counter variable for material flows/product types
mf_max = sym.symbols('mf_max') # total number of material flows / product types
P = sym.symbols('P') # penalty value
t = sym.symbols('t') # counter variable for time ticks/slices
T = sym.symbols('T') # max number of slices for any product
t_max = sym.IndexedBase('t_max') # max number of slices for respective product
len_t = sym.IndexedBase('len_t') # length of time tick / number of vertices in given tick

u = sym.IndexedBase('u') # used to access array of unique vertices
len_u = sym.symbols('len_u') # number of unique vertices

d = sym.IndexedBase('d') # represents the function d, which returns the edge cost between two vertices
D = sym.IndexedBase('D') # represents the function D_function (see above)

s = sym.IndexedBase('s') # variable for sliced material flows
ns = sym.IndexedBase('ns') # variable for numbered and sliced material flows



## Definition of the QUBO/Cost Fuction

In [17]:
# 1st Constraint: Each time tick of each material has exactly one active machine.
# Multiplications by 0.5 and 2 are conducted to avoid an incorrect definition of the constraint by sympy.

constraint_1 = ((0.5*P*sym.Sum(
                      (2*sym.Sum(
                          (sym.Sum(X[ns[k,t,v]],            
                              (v, 0, len_t[k,t] - 1))-1)**2, 
                          (t, 0, t_max[k]-1))), 
                      (k, 0, mf_max-1))))
constraint_1

0.5*P*Sum(2*Sum((Sum(X[ns[k, t, v]], (v, 0, len_t[k, t] - 1)) - 1)**2, (t, 0, t_max[k] - 1)), (k, 0, mf_max - 1))

In [18]:
# 2nd Constraint: Each machine can only execute one operation at once.
# The intervals [c=1,c_max-2] and [v=1,len_u-2] are chosen, since both, start and destination nodes, 
# do not need to be considered.

constraint_2 = (P * sym.Sum( 
                      sym.Sum(((D[u[v],t,mf_max]**2 - D[u[v],t,mf_max])) , 
                          (v, 1, len_u-2)), 
                      (t, 1, T - 2)))
constraint_2

P*Sum(D[u[v], t, mf_max]**2 - D[u[v], t, mf_max], (v, 1, len_u - 2), (t, 1, T - 2))

In [19]:
# Main cost function: Costs for the chosen machines/paths is summed up.
# Similar as before, effectless multiplications (here: 0.25*2*2 = 1) need to be conducted to avoid an incorrect 
# definition by sympy.

cost_function_main = (0.25*sym.Sum(   
                        (2*sym.Sum(  
                            (2*sym.Sum(X[ns[k,t,v]] * X[ns[k,t+1,w]] * d[s[k,t,v], s[k,t+1,w]],  
                                    (w, 0, len_t[k,t + 1] - 1) , (v, 0, len_t[k,t] - 1))),
                            (t, 0, t_max[k] - 2))),
                     (k, 0, mf_max-1)))
cost_function_main

0.25*Sum(2*Sum(2*Sum(X[ns[k, t + 1, w]]*X[ns[k, t, v]]*d[s[k, t, v], s[k, t + 1, w]], (w, 0, len_t[k, t + 1] - 1), (v, 0, len_t[k, t] - 1)), (t, 0, t_max[k] - 2)), (k, 0, mf_max - 1))

In [20]:
# Eventually, the final cost function, consisting of the main cost function and both constraints, is defined as a sum of all.

cost_function = constraint_1 + constraint_2 + cost_function_main
cost_function

0.5*P*Sum(2*Sum((Sum(X[ns[k, t, v]], (v, 0, len_t[k, t] - 1)) - 1)**2, (t, 0, t_max[k] - 1)), (k, 0, mf_max - 1)) + P*Sum(D[u[v], t, mf_max]**2 - D[u[v], t, mf_max], (v, 1, len_u - 2), (t, 1, T - 2)) + 0.25*Sum(2*Sum(2*Sum(X[ns[k, t + 1, w]]*X[ns[k, t, v]]*d[s[k, t, v], s[k, t + 1, w]], (w, 0, len_t[k, t + 1] - 1), (v, 0, len_t[k, t] - 1)), (t, 0, t_max[k] - 2)), (k, 0, mf_max - 1))

In [21]:
# Translation of data into dictionaries for sympy.

# Dictionary that include values that have a fixed value
single_valued_dict = {
    mf_max: nr_flows,
    P: penalty_value,
    T: highest_slice_nr,
    len_u: len(unique_nodes)
    }

# Dictionary for the number of slices/time ticks per material flow
nr_slices_dict = {
    t_max[k]: len(mf_s[k]) for k in range(nr_flows)
}

# Dictionary for the unique nodes in the MMS. In the given examples, these are:
# 0 = Source, 1 = M1, 2 = M2, 3 = M3, 4 = M4, 5 = M5, 6 = M6, 7 = Destination  
unique_nodes_dict = {
    u[v]: unique_nodes[v] for v in range(len(unique_nodes))
}

# Dictionary for the numbered nodes in the system 
# e.g. node 10 is equal to ns[1][1][1] (material flow 2, operation 1 on time tick t_1, machine 3)
numbered_sliced_MF_dict = {
    ns[k, i, j]: mf_ns[k][i][j] for k in range(nr_flows) for i in range(len(mf_ns[k])) for j in range(len(mf_ns[k][i]))
}


# Dictionary for the function D that controls for collisions of material flows
D_function_dict = {
    D[v, t, r]: D_function(v, t, r) for v in range(len(unique_nodes)) for t in range(highest_slice_nr) for r in range(nr_flows, nr_flows+1)
}

# Dictionary for the nodes (source/machines/destination nodes) in general.
# Other than numbered_sliced_MF_dict, this dictionary assigns the total nodes of all material flows to the initial numbers 0-7. 
sliced_MF_dict = {
    s[k, i, j]: mf_s[k][i][j] for k in range(nr_flows) for i in range(len(mf_s[k])) for j in range(len(mf_s[k][i]))
}

# Dictionary for the length of the slice/time tick the function currently looks at.
# For the given problem formulation the lenth is either 1 (t_0/t_4 for source/destination) or two (t_1-t_3 for operations).
len_slice_dict = {
    len_t[k, i]: len(mf_s[k][i]) for k in range(nr_flows) for i in range(len(mf_s[k]))
}

# Dictionary for the cost/weight of choosing one route
d_dict = {
    d[i, j]: edge_cost_matrix[i][j] 
    for i in unique_nodes
    for j in unique_nodes
}

# Definition of the cost polynomial of the optimization problem
cost_poly = sym.Poly(cost_function
                     .subs(single_valued_dict)
                     .doit()
                     .subs(nr_slices_dict)
                     .doit()
                     .subs(unique_nodes_dict)
                     .doit()
                     .subs(len_slice_dict)
                     .doit()
                     .subs(D_function_dict)
                     .doit()
                     .subs(numbered_sliced_MF_dict)
                     .subs(sliced_MF_dict)
                     .doit()
                     .subs(d_dict)
                     .doit(),
                     [X[i] for i in range(nr_qubits)])
cost_poly

# The only variables in the cost polynomial should be X_0, X_1, X_2 etc. if not, then some of the variables have not been evaluated; order of evaluation is important.
# If the evaluation of parts of the polynomial depends on the evaluation of another, then doit() is necessary in between in order to evaluate that first part.

Poly(200.0*X[0]**2 + 1.0*X[0]*X[1] + 1.0*X[0]*X[2] - 400.0*X[0] + 400.0*X[1]**2 + 400.0*X[1]*X[2] + 1.0*X[1]*X[3] + 2.0*X[1]*X[4] - 600.0*X[1] + 400.0*X[2]**2 + 2.0*X[2]*X[3] + 1.0*X[2]*X[4] + 400.0*X[2]*X[9] - 600.0*X[2] + 400.0*X[3]**2 + 400.0*X[3]*X[4] + 1.0*X[3]*X[5] + 2.0*X[3]*X[6] - 600.0*X[3] + 400.0*X[4]**2 + 2.0*X[4]*X[5] + 1.0*X[4]*X[6] - 600.0*X[4] + 400.0*X[5]**2 + 400.0*X[5]*X[6] + 1.0*X[5]*X[7] + 400.0*X[5]*X[14] - 600.0*X[5] + 400.0*X[6]**2 + 1.0*X[6]*X[7] - 600.0*X[6] + 200.0*X[7]**2 - 400.0*X[7] + 200.0*X[8]**2 + 1.0*X[8]*X[9] + 3.0*X[8]*X[10] - 400.0*X[8] + 400.0*X[9]**2 + 400.0*X[9]*X[10] + 1.0*X[9]*X[11] + 3.0*X[9]*X[12] - 600.0*X[9] + 400.0*X[10]**2 + 1.0*X[10]*X[11] + 2.0*X[10]*X[12] - 600.0*X[10] + 400.0*X[11]**2 + 400.0*X[11]*X[12] + 2.0*X[11]*X[13] + 3.0*X[11]*X[14] - 600.0*X[11] + 400.0*X[12]**2 + 1.0*X[12]*X[13] + 1.0*X[12]*X[14] - 600.0*X[12] + 400.0*X[13]**2 + 400.0*X[13]*X[14] + 3.0*X[13]*X[15] - 600.0*X[13] + 400.0*X[14]**2 + 1.0*X[14]*X[15] - 600.0*X[14]

In [22]:
# The offset of the above definded cost function is not included in the cost function that will be passed to the algorithm.
# So, it will be added later to get the actual result (see line 'OPTIMIZATION COST' in section 'Results')
offset = cost_poly.coeff_monomial(1)

In [23]:
# Generate Qiskit's cost function by defining a new QuadraticProgram()
qiskit_cost_function = QuadraticProgram()

# Define Qiskit variables
for i in range(nr_qubits):                                                                     # binary variables x0 - xn
    qiskit_cost_function.binary_var('X_' + str(i))
    
# With the following line of code a binary variable representing the offset could be added to the function     
#   qiskit_cost_function.integer_var(lowerbound=int(offset),upperbound=int(offset), name='o')  # integer variable: o=offset
# The optimization algorithm will run and deliver valid results, but the problem of the final result representation 
# could not be solved.

# Specification of the final Qiskit cost function
# Here, the function is assigned its linear and quadratic parts using the above defined variables.
qiskit_cost_function.minimize(
    linear = [int(cost_poly.coeff_monomial(X[i]**1)) for i in range(nr_qubits)],
    quadratic = {
        ('X_'+str(i), 'X_'+str(j)): cost_poly.coeff_monomial(X[i]**1 * X[j]**1)
        for i in range(nr_qubits)
        for j in range(i,nr_qubits)
    }
    )
for i in range(nr_flows):
    fix_nodes(start_node, dest_node, i, mf_s[i], mf_ns[i], qiskit_cost_function)

# The final Qiskit cost function is printed and can be controlled after executing this piece of code.
print(qiskit_cost_function.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 400 X_0 - 600 X_1 - 600 X_2 - 600 X_3 - 600 X_4 - 600 X_5 - 600 X_6
      - 400 X_7 - 400 X_8 - 600 X_9 - 600 X_10 - 600 X_11 - 600 X_12 - 600 X_13
      - 600 X_14 - 400 X_15 + [ 400 X_0^2 + 2 X_0*X_1 + 2 X_0*X_2 + 800 X_1^2
      + 800 X_1*X_2 + 2 X_1*X_3 + 4 X_1*X_4 + 800 X_2^2 + 4 X_2*X_3 + 2 X_2*X_4
      + 800 X_2*X_9 + 800 X_3^2 + 800 X_3*X_4 + 2 X_3*X_5 + 4 X_3*X_6
      + 800 X_4^2 + 4 X_4*X_5 + 2 X_4*X_6 + 800 X_5^2 + 800 X_5*X_6 + 2 X_5*X_7
      + 800 X_5*X_14 + 800 X_6^2 + 2 X_6*X_7 + 400 X_7^2 + 400 X_8^2 + 2 X_8*X_9
      + 6 X_8*X_10 + 800 X_9^2 + 800 X_9*X_10 + 2 X_9*X_11 + 6 X_9*X_12
      + 800 X_10^2 + 2 X_10*X_11 + 4 X_10*X_12 + 800 X_11^2 + 800 X_11*X_12
      + 4 X_11*X_13 + 6 X_11*X_14 + 800 X_12^2 + 2 X_12*X_13 + 2 X_12*X_14
      + 800 X_13^2 + 800 X_13*X_14 + 6 X_13*X_15 + 800 X_14^2 + 2 X_14*X_15
      + 400 X_15^2 ]/2
Subject To
 start_node_veh_0: X_0 = 1


## Initialization of QAOA and Classical Optimizer

In [24]:
# The QAOA is executed on a local Qasm Simulator.
# COBYLA, SPSA, and others are admissible optimizers to solve a systems of multiple unknown parameters.
# To receive more/less accurate results, adjustments can be made on the parameters maxiter, reps and shots.
maxiter = 10
optimizer = COBYLA(maxiter=maxiter) # Typical classical optimizerts to use are e.g. COBYLA, L_BFGS_B, POWELL, SLSQP, SPSA
backend = qiskit.Aer.get_backend('qasm_simulator')
qaoa = QAOA(reps=10, optimizer=optimizer, quantum_instance =
             QuantumInstance(backend=backend, shots = 8000, skip_qobj_validation=False))

# MinimumEigenOptimizer eventually forms the QUBO in binary form and thereof into an Ising Hamiltionian that is solved 
optimizer_qaoa = MinimumEigenOptimizer(qaoa)

# Empty list to save the results of all iterations.
results = []

# Time list, in which the starting time of the for-loop is registered in ticks[0].
# Hence, a calculation of one iteration's processing time is easily determinable later. 
ticks = [time.time()]

# With the library tqdm we can create a progress bar to check the time progress of the iterations (it).
# It is structured as follows: progress in % | progress bar | relative progress [it start time < total remaining time, s/it].
for i in tqdm(range(10)):
    result_qaoa = optimizer_qaoa.solve(qiskit_cost_function)
    results.append(result_qaoa)
    print(results[i])    
    ticks.append(time.time())

 10%|████████▎                                                                          | 1/10 [00:56<08:30, 56.75s/it]

fval=-1989.0, X_0=1.0, X_1=1.0, X_2=0.0, X_3=1.0, X_4=0.0, X_5=1.0, X_6=0.0, X_7=1.0, X_8=1.0, X_9=1.0, X_10=0.0, X_11=1.0, X_12=0.0, X_13=1.0, X_14=0.0, X_15=1.0, status=SUCCESS


 20%|████████████████▌                                                                  | 2/10 [01:52<07:27, 55.92s/it]

fval=-1989.0, X_0=1.0, X_1=1.0, X_2=0.0, X_3=0.0, X_4=1.0, X_5=0.0, X_6=1.0, X_7=1.0, X_8=1.0, X_9=1.0, X_10=0.0, X_11=1.0, X_12=0.0, X_13=0.0, X_14=1.0, X_15=1.0, status=SUCCESS


 30%|████████████████████████▉                                                          | 3/10 [02:48<06:32, 56.13s/it]

...

fval=-1989.0, X_0=1.0, X_1=1.0, X_2=0.0, X_3=1.0, X_4=0.0, X_5=0.0, X_6=1.0, X_7=1.0, X_8=1.0, X_9=1.0, X_10=0.0, X_11=1.0, X_12=0.0, X_13=0.0, X_14=1.0, X_15=1.0, status=SUCCESS


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [08:24<00:56, 56.21s/it]

fval=-1989.0, X_0=1.0, X_1=1.0, X_2=0.0, X_3=0.0, X_4=1.0, X_5=0.0, X_6=1.0, X_7=1.0, X_8=1.0, X_9=1.0, X_10=0.0, X_11=0.0, X_12=1.0, X_13=0.0, X_14=1.0, X_15=1.0, status=SUCCESS


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [09:21<00:00, 56.14s/it]

fval=-1988.0, X_0=1.0, X_1=1.0, X_2=0.0, X_3=1.0, X_4=0.0, X_5=1.0, X_6=0.0, X_7=1.0, X_8=1.0, X_9=1.0, X_10=0.0, X_11=0.0, X_12=1.0, X_13=1.0, X_14=0.0, X_15=1.0, status=SUCCESS


## Results

In [25]:
# Representation of the results above in a table including the optimization cost of a respective path chosen,
# the material flow path in qubit as well as real forn and a validity check.
        
# Data frame to visualize final results
result_df = pd.DataFrame(columns = ['opt. cost', 'path', 'real path', 'validity', 'sim. time'])
#result_df = pd.DataFrame({'opt. cost': [opt_cost], 'path': [qubit_path], 'real path': [real_path], 'validity': [validity], 'sim. time': [sim_time]})

# FOR-LOOP: all result values (opt. cost, qubit/real path, validity) are determined. 
t = 0
sim_t = []

for r in results:
    
    # OPTIMIZATION COST
    opt_cost = round(r.fval + offset) 
    
    # VALIDITY [val = valid; inv = invalid]
    if (r.fval + offset) > penalty_value:
        validity = "inv"
    else:
        validity = "val"
    
    # SIMULATION TIME
    sim_time = str(round(ticks[t+1] - ticks[t], 2)) + ' s'
    sim_t.append(sim_time)
    
    t += 1
    
    # QUBIT PATH consisting of 8*n qubit representations with n representing the total number of material flows / jobs.
    
    qubit_path = str(r.x).replace(' ', '').replace('.', '')[1:-1]   # Create string that only includes path numbers 
    
    if nr_flows == 2:    # Final formatting of string for 2 material flows
        qubit_path = "{}-{}-{}-{}-{} | {}-{}-{}-{}-{}".format(qubit_path[0], qubit_path[1:3], qubit_path[3:5], qubit_path[5:7], 
                     qubit_path[7], qubit_path[8], qubit_path[9:11], qubit_path[11:13], qubit_path[13:15], qubit_path[15])
        
    elif nr_flows == 3:   # Final formatting of string for 3 material flows
        qubit_path = "{}-{}-{}-{}-{} | {}-{}-{}-{}-{} | {}-{}-{}-{}-{}".format(qubit_path[0], qubit_path[1:3], qubit_path[3:5], qubit_path[5:7], 
                     qubit_path[7], qubit_path[8], qubit_path[9:11], qubit_path[11:13], qubit_path[13:15], qubit_path[15], 
                     qubit_path[16], qubit_path[17:19], qubit_path[19:21], qubit_path[21:23], qubit_path[23])
        
    else:                 # Final formatting of string for 4+ material flows -> output: sequence of 8 bits * nr_flows
        continue          #                     --- limitedly supported by qasm simulator ---
    
    
    # REAL PATH consisting of 5*n machines with n being the total number of material flows / jobs
    
    real_path = list(str(r.x).replace(' ', '').replace('.', '')[1:-1])  # Transfer result string into list to allow formatting
    
    for i in range(1, len(real_path)+1): 
        if int(i)%8 == 0:                         # Assigns destination marks
            real_path[i-1] = 'D'
        elif (int(i)%8)-1 == 0 or i == 1:             # Assigns source marks
            real_path[i-1] = 'S'
        elif real_path[i-1] == '1':               # Assigns machine to active qubit (qubit value = 1)
            real_path[i-1] = str(mf_s_list[i-1])
        else:                                     # Deletes inactive qubits (qubit value = 0)
            real_path[i-1] = ""

    real_path = "".join(real_path).replace('','-')[1:-1]    # Join list elements to form string
    
    if nr_flows == 2:                                       # Final formatting of string for 2 material flows
        real_path = real_path[:9] + ' | ' + real_path[10:]  
    elif nr_flows == 3:                                     # Final formatting of string for 3 material flows
        real_path = real_path[:9] + ' | ' + real_path[10:19] + ' | ' + real_path[20:]
    else:                                                   # Final formatting of string for 4+ material flows
        continue                                            # --- limitedly supported by qasm simulator ---
    
    
    # APPEND RESULTS of iteration to data frame
    result_df = result_df.append({'opt. cost': opt_cost, 'path': qubit_path, 'real path': real_path, 
                                  'validity': validity, 'sim. time':  sim_time}, ignore_index=True)
    #new_row = pd.DataFrame({'opt. cost': [opt_cost], 'path': [qubit_path], 'real path': [real_path], 'validity': [validity], 'sim. time': [sim_time]})
    #result_df = pd.concat([result_df, new_row], ignore_index=True)
    
# PRINT final results sorted by the optimization cost from best to worst
#print("QAOA:")
#print(result_df.sort_values(by=['opt. cost']))

from IPython.display import display, HTML
display(HTML(result_df.sort_values(by=['opt. cost']).to_html()))

C:\Users\Rapha\AppData\Local\Temp\ipykernel_11632\2808545056.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'opt. cost': opt_cost, 'path': qubit_path, 'real path': real_path,
C:\Users\Rapha\AppData\Local\Temp\ipykernel_11632\2808545056.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'opt. cost': opt_cost, 'path': qubit_path, 'real path': real_path,
C:\Users\Rapha\AppData\Local\Temp\ipykernel_11632\2808545056.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'opt. cost': opt_cost, 'path': qubit_path, 'real path': real_path,
C:\Users\Rapha\AppData\Local\Temp\ipykernel_11632\2808545056.py:71: FutureWarning: The frame.append 

,opt. cost,path,real path,validity,sim. time
0,11,1-10-10-10-1 | 1-10-10-10-1,S-1-3-5-D | S-2-1-4-D,val,56.75 s
1,11,1-10-01-01-1 | 1-10-10-01-1,S-1-4-6-D | S-2-1-5-D,val,55.34 s
2,11,1-01-01-01-1 | 1-01-01-01-1,S-2-4-6-D | S-3-6-5-D,val,56.38 s
3,11,1-10-01-01-1 | 1-10-01-01-1,S-1-4-6-D | S-2-6-5-D,val,55.79 s
4,11,1-10-10-10-1 | 1-10-10-10-1,S-1-3-5-D | S-2-1-4-D,val,55.3 s
6,11,1-10-10-10-1 | 1-10-10-10-1,S-1-3-5-D | S-2-1-4-D,val,57.04 s
7,11,1-10-10-01-1 | 1-10-10-01-1,S-1-3-6-D | S-2-1-5-D,val,56.53 s
8,11,1-10-01-01-1 | 1-10-01-01-1,S-1-4-6-D | S-2-6-5-D,val,56.02 s
9,12,1-10-10-10-1 | 1-10-01-10-1,S-1-3-5-D | S-2-6-4-D,val,56.43 s
5,13,1-10-10-01-1 | 1-10-01-10-1,S-1-3-6-D | S-2-6-4-D,val,55.8 s


In the table above, the results of the last 10 calculations can be found. A respective result is split up into the following:
- An optimization cost, which is built up acc. to the cost function described earlier. The cost for active edges between nodes is defined in the material flow edge_cost_matrix.
- A path that shows the qubit measurements at the end of the optimization cycle. As described earlier, 1's are representing nodes that are active in specific time ticks. Depending on the number of material flows, there are 16, 24 or more binary digits. A string of 8 digits separated by "-" for every time tick (e.g. 1-10-10-10-1) represents one material flow.
- A real path that takes the path results and interprets them as true results. So, each time tick is assigned with an actual node (source, machine or destination) in the problem. For example, the path string '1-10-10-10-1' is representing the real path 'S-1-3-5-D' for material flow 1, and the path string '1-01-01-10-1' is representing the real path 'S-3-6-4-D' for material flow 2, as they were defined in mf_s in the preprocessing section.
- A validity check that tells if the calculated route is valid or invalid.
- The simulation time that was needed to process the optimization problem in the given iteration. 

## Initialization of VQE Algorithm and Classical Optimizer [not included in the project]

In [27]:
# Taken from Madadi (2021) and modified to handle the requirements of the result representation, the following code 
# can be used to compute the problem using the VQE algorithm. Please use the code cell in section "Results" to display 
# the results gained out of the following simulation runs. 

from qiskit.algorithms import VQE
from qiskit.circuit.library import TwoLocal

backend = qiskit.Aer.get_backend('qasm_simulator')
optimizer = COBYLA(maxiter=10)
ry = TwoLocal(nr_qubits, 'ry', 'cz', reps=10, entanglement='linear') # ansatz

vqe = VQE(ry, optimizer=optimizer, quantum_instance=QuantumInstance(backend=backend, shots = 8000))

optimizer_vqe = MinimumEigenOptimizer(vqe)

results = []
ticks = [time.time()]

for i in tqdm(range(10)):
    results_VQE = optimizer_vqe.solve(qiskit_cost_function)
    results.append(results_VQE)
    print(results[i]) 
    ticks.append(time.time())

 10%|████████▎                                                                          | 1/10 [00:44<06:42, 44.72s/it]

fval=-1793.0, X_0=1.0, X_1=1.0, X_2=0.0, X_3=0.0, X_4=1.0, X_5=0.0, X_6=1.0, X_7=1.0, X_8=1.0, X_9=0.0, X_10=0.0, X_11=0.0, X_12=1.0, X_13=0.0, X_14=1.0, X_15=1.0, status=SUCCESS


 20%|████████████████▌                                                                  | 2/10 [01:16<04:58, 37.33s/it]

fval=-1988.0, X_0=1.0, X_1=1.0, X_2=0.0, X_3=0.0, X_4=1.0, X_5=0.0, X_6=1.0, X_7=1.0, X_8=1.0, X_9=1.0, X_10=0.0, X_11=1.0, X_12=0.0, X_13=1.0, X_14=0.0, X_15=1.0, status=SUCCESS


 30%|████████████████████████▉                                                          | 3/10 [01:54<04:22, 37.44s/it]

...

fval=-1988.0, X_0=1.0, X_1=1.0, X_2=0.0, X_3=1.0, X_4=0.0, X_5=0.0, X_6=1.0, X_7=1.0, X_8=1.0, X_9=0.0, X_10=1.0, X_11=0.0, X_12=1.0, X_13=0.0, X_14=1.0, X_15=1.0, status=SUCCESS


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [05:03<00:30, 30.60s/it]

fval=-1989.0, X_0=1.0, X_1=1.0, X_2=0.0, X_3=1.0, X_4=0.0, X_5=1.0, X_6=0.0, X_7=1.0, X_8=1.0, X_9=1.0, X_10=0.0, X_11=1.0, X_12=0.0, X_13=1.0, X_14=0.0, X_15=1.0, status=SUCCESS


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [05:34<00:00, 33.41s/it]

fval=-1786.0, X_0=1.0, X_1=1.0, X_2=0.0, X_3=1.0, X_4=1.0, X_5=0.0, X_6=1.0, X_7=1.0, X_8=1.0, X_9=1.0, X_10=0.0, X_11=1.0, X_12=0.0, X_13=0.0, X_14=1.0, X_15=1.0, status=SUCCESS


## Sources

- Jaroszewski, D., Klos, F., & Sturm, B. (2020). Ising formulations of routing optimization problems. Retrieved from: https://doi.org/10.48550/arXiv.2012.05022.
- Klos, F. (2020). Routing. PlanQK. Retrieved from: https://github.com/PlanQK/Routing.
- Madadi, L. (2021). Application of Quantum Computing to Use Cases from Travel & Transport Industries. Retrieved from: https://planqk.stoneone.de/veroeffentlichungen/.
- Qiskit Development Team. (2023a). Qiskit Textbook. Retrieved from: https://qiskit.org/documentation/.
- Qiskit Development Team. (2023b). QuantumInstance. Retrieved from: https://qiskit.org/documentation/stubs/qiskit.utils.QuantumInstance.html.